In [ ]:
# !pip install autogen

In [1]:
import os
import re
import io
import logging
import contextlib
import pandas as pd
from autogen import AssistantAgent, UserProxyAgent

/opt/conda/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [ ]:
# Suppress autogen logs
logging.getLogger("autogen").setLevel(logging.ERROR)

# Set your Fireworks API key and base URL
os.environ["FIREWORKS_API_KEY"] = "YOUR_FIREWORKS_API_KEY"
os.environ["FIREWORKS_API_BASE"] = "https://api.fireworks.ai/inference/v1"

# Fireworks LLM config
llm_config = {
    "config_list": [
        {
            "api_key": os.getenv("FIREWORKS_API_KEY"),
            "base_url": os.getenv("FIREWORKS_API_BASE"),
            "model": "accounts/fireworks/models/llama-v3p1-405b-instruct",
        }
    ],
    "temperature": 0.7,
    "max_tokens": 5
}

In [3]:
fitness_agent = AssistantAgent(
    name = 'FitnessAgent',
    llm_config = llm_config,
    system_message= """You are a classification model, your task is to classify the given text into two categories: 'interested' or 'not_interested'.
          The text will primarily be in Romanized Hindi and English Language, your goal is to identify the intent of the user typing these sentences as, if the user is interested in health and fitness or not.
          **Guidlines for evaluating the results :**
          1. If the text contains the intent of health and fitness, in this case classify as 'interested'.
          2. If the text does not contain the intent of health and fitness, in this case classify as 'not_interested'.
          
          Your goal is to evaluate the overall context, combining various indicators to determine the most likely intent, without being overly influenced by individual ambiguous terms. 
          Make sure to give 'interested' label only when the context of the sentence is of sports or health or fitness, no other intent except for the mentioned are to be given 'interested' label.
          
          Some examples for this classification are as follows:-
          1. 'i am completing my maths exercise' : In this sentence, the user is talking about solving maths problem and completing some academic exercise and not a physical exercise nor workout nor any fitness related activity, in this case, label should be 'not_interested'.
          2. 'yoga cake is good': In this sentence, user is talking about a type of cake, and not specifically talking about yoga as a exercise. In this case the label should be 'not_interested'.
          3. 'i am going for my workout': In this sentence, user is telling that he is going for workout. In this case, the label should be 'interested'.
          4. 'rd exercise', 'rs aggarwal se exercise practice krli hai': Inthese two sentences, user is not saying about a physical exercise rather using shortforms of some books or teachers whose assignment is to be completed referring to as exercise. In these cases, label should be 'not_interested'.
          
          
          Respond with only the given labels, without any additional commentary. Make sure to use the given examples for classifying the correct intent of the sentence.
          If you cannot classify the text with certainty, respond with 'not_interested'."""
)

marital_agent = AssistantAgent(
    name = 'MaritalAgent',
    llm_config = llm_config,
    system_message = """You are a classification model. Your task is to predict my relationship status out of the given options: "married", "divorced", "in_a_relationship", "in_a_situationship", "uncertain".
                The text will primarily be in Romanized Hindi and English. Your goal is to identify my relationship status. I may also talk about other people; do not use such sentences to give a positive label about my relationship status.
                If I explicitly say something about myself, only then assign a positive label from the given options. If unsure whether I am talking about myself, respond with the "uncertain" label.

                Examples of sentences that should be used to understand the context of the sentences:

                **married:**
                1. "I have a wife and two children." - Label: "married".
                2. "Meri wife ghar par wait kr rhi hogi." - Label: "married".
                3. "Hamari shaadi ko 5 saal ho gaye hain." - Label: "married".
                4. "Main aur meri wife weekend par trip par ja rahe hain." - Label: "married".
                5. "Meri shaadi ek arranged marriage thi." - Label: "married".

                **divorced:**
                1. "I got separated from my wife years ago." - Label: "divorced".
                2. "Mera divorce case chal rha hai." - Label: "divorced".
                3. "Main apni purani shaadi ke bare mein sochta hoon." - Label: "divorced".
                4. "Meri wife se divorce finalize ho gaya hai." - Label: "divorced".
                5. "Hum dono ka alag hona better decision tha." - Label: "divorced".

                **in_a_relationship:**
                1. "I am going out on a date tomorrow." - Label: "in_a_relationship".
                2. "Bhai vo meri wife ke liye gift dekh rha hu koi." - Label: "in_a_relationship".
                3. "Main aur meri girlfriend kal movie dekhne ja rahe hain." - Label: "in_a_relationship".
                4. "I just proposed to my partner, and she said yes." - Label: "in_a_relationship".
                5. "Mujhe apni girlfriend ke birthday ke liye surprise plan karna hai." - Label: "in_a_relationship".

                **in_a_situationship:**
                1. "I am not sure about my relationship with her; we kind of like each other, but I am not ready for a commitment." - Label: "in_a_situationship".
                2. "Bhai abhi committed nhi hu main, situationship samajh le isko." - Label: "in_a_situationship".
                3. "Hum dono ek dusre ko pasand karte hain, but kuch officially decide nahi hua hai." - Label: "in_a_situationship".
                4. "I like her, but I don't know if this will turn into something serious." - Label: "in_a_situationship".
                5. "She and I are figuring things out, but nothing is defined yet." - Label: "in_a_situationship".

                **uncertain:**
                1. "Today for the first time I am seeing someone so busy with songs." - Label: "uncertain".
                2. "Kal chalte hai bhai college." - Label: "uncertain".
                3. "Mujhe pata nahi main aaj kal kis direction mein hoon." - Label: "uncertain".
                4. "He is dating someone; I wonder how that feels." - Label: "uncertain".
                5. "My friend said he's not sure about his relationship, so I don't know what to tell him." - Label: "uncertain".

                Your goal is to evaluate the overall context, combining various indicators to determine my relationship status, without being overly influenced by ambiguous terms. Make sure you understand the context with a good degree of certainty and then give the output label.
                Respond with only the given labels, without any additional commentary. If you cannot classify the text with certainty, respond with the "uncertain" label.
                """

)

# Create user proxy agent
user_proxy = UserProxyAgent(
    name="UserProxy",
    is_termination_msg=lambda msg: "exit" in msg.get("content", "").lower(),
    human_input_mode="NEVER"
)


# Agent selector
agent_map = {
    "fitness": fitness_agent,
    "marital": marital_agent
}

In [4]:
df = pd.read_csv('sample_results_on_multi_ai_agent_system.csv', usecols=['gaid', 'sentence'])

In [5]:
df

,gaid,sentence
0,0430a06e-7853-472c-aeae-b79b3e52da3d,"['hot water bag chala', 'correct ga vache mund..."
1,0539e7b3-3bf1-49bc-b34e-c87603f71077,['mera dimak khrb mt kro smjhe tum nhi hu m us...
2,05cff5ea-9742-4f9f-a70c-18642ffbdc25,['gym hu okh']
3,07d9021d-5166-494a-8ed2-4293f11a0e5d,['hamare wife bhi rhegi sath me']
4,07ff08af-d565-4fff-b2e6-d4a36246cbe7,"['running', 'running shorts men', 'running sho..."
...,...,...
95,fc7ae7eb-82fd-4693-90ba-24b2e36a2d06,['pkaa meri wife ne bola']
96,fcf41be4-2126-49b8-873c-d1ae21f25cfa,['my wife']
97,fd78226a-5f09-43da-b7c6-c1b5dae0a388,['lenovo yoga tab']
98,fe2db4ee-c331-4fff-aab0-aa00118e1a54,"['swimming poo', 'swimming pool mom son', 'swi..."


In [19]:
# Main function
def classify_sentences():
    final_health_label_list = []
    final_marital_label_list = []
    
    sentences = df['sentence'].tolist()[:5]

    # Store results for clean final output
    final_results = []
    
     # Run classification
    for idx, sentence in enumerate(sentences):
        result = {}
        
        for agent_key in ['fitness', 'marital']:
            agent = agent_map.get(agent_key)
            
            response = user_proxy.initiate_chat(
                agent,
                message=f"The sentence to be classified is: {sentence}",
                max_turns=1,
                summary_method=None
            )
            
            reply = response.chat_history[-1]["content"].strip().lower()
            
            if agent_key == 'fitness':
                final_health_label_list.append(reply)
                
            else:
                final_marital_label_list.append(reply)
                
#     result_df = pd.DataFrame()
#     result_df['gaid'] = df['gaid'].tolist()[:5]
#     result_df['sentence'] = df['sentence'].tolist()[:5]
#     result_df['health_results'] = final_health_label_list
#     result_df['marital_results'] = final_marital_label_list
    
    return final_health_label_list, final_marital_label_list

In [20]:
if __name__ == "__main__":
    list1, list2 = classify_sentences()

UserProxy (to FitnessAgent):

The sentence to be classified is: ['hot water bag chala', 'correct ga vache mundu roje ram naku hot water bag konichadu']

--------------------------------------------------------------------------------
FitnessAgent (to UserProxy):

interested

--------------------------------------------------------------------------------
UserProxy (to MaritalAgent):

The sentence to be classified is: ['hot water bag chala', 'correct ga vache mundu roje ram naku hot water bag konichadu']

--------------------------------------------------------------------------------
MaritalAgent (to UserProxy):

uncertain

--------------------------------------------------------------------------------
UserProxy (to FitnessAgent):

The sentence to be classified is: ['mera dimak khrb mt kro smjhe tum nhi hu m usse relationship me or hn agr abi bi yakin na hoto jake puch lo kisi or se']

--------------------------------------------------------------------------------
FitnessAgent (to Us

In [23]:
result_df = pd.DataFrame()
result_df['gaid'] = df['gaid'].tolist()[:5]
result_df['sentence'] = df['sentence'].tolist()[:5]
result_df['health_results'] = list1
result_df['marital_results'] = list2

In [24]:
result_df

,gaid,sentence,health_results,marital_results
0,0430a06e-7853-472c-aeae-b79b3e52da3d,"['hot water bag chala', 'correct ga vache mund...",interested,uncertain
1,0539e7b3-3bf1-49bc-b34e-c87603f71077,['mera dimak khrb mt kro smjhe tum nhi hu m us...,not_interested,in_a_relationship
2,05cff5ea-9742-4f9f-a70c-18642ffbdc25,['gym hu okh'],interested,uncertain
3,07d9021d-5166-494a-8ed2-4293f11a0e5d,['hamare wife bhi rhegi sath me'],not_interested,married
4,07ff08af-d565-4fff-b2e6-d4a36246cbe7,"['running', 'running shorts men', 'running sho...",interested,uncertain


In [25]:
result_df.to_csv('initial_pipeline_results.csv', index=False)

#### Custom Input

In [7]:
# Main function
def classify_sentences():
    print("Enter sentences (comma-separated):")
    sentences = [s.strip() for s in input("Sentences: ").split(",") if s.strip()]
    
    print("\nSpecify agent(s) (e.g., sports, education or sports|fitness for multiple agents per sentence):")
    print("  - If same agent(s) for all, just provide one set (e.g., sports or sports|fitness)")
    print("  - If per-sentence agents, match count of agents to number of sentences (comma-separated)")
    agent_input = input("Agent(s): ").strip()

    if "|" in agent_input or "," in agent_input:
        # Either multiple agents per sentence or per-sentence mapping
        agent_lists = [a.strip().split("|") for a in agent_input.split(",")]
        if len(agent_lists) == 1:
            agent_lists = [agent_lists[0]] * len(sentences)
        elif len(agent_lists) != len(sentences):
            print("Mismatch between number of sentences and agent groups.")
            return
    else:
        # One agent for all
        agent_lists = [[agent_input.strip().lower()]] * len(sentences)

    # Store results for clean final output
    final_results = []

    # Run classification
    for idx, (sentence, agent_keys) in enumerate(zip(sentences, agent_lists), 1):
        result = {}
        for agent_key in agent_keys:
            agent_key = agent_key.lower()
            agent = agent_map.get(agent_key)
            if agent:
                f = io.StringIO()
                with contextlib.redirect_stdout(f):
                    response = user_proxy.initiate_chat(
                        agent,
                        message=f"The sentence to be classified is: {sentence}",
                        max_turns=1,
                        summary_method=None
                    )
                reply = response.chat_history[-1]["content"].strip().lower()
                result[agent_key] = reply
            else:
                result[agent_key] = "invalid_agent"
        
        final_results.append({
            "sentence": sentence,
            "agents": "|".join(agent_keys),
            "results": result
        })

    print("\n" + "=" * 60 + "\n")
    for entry in final_results:
        print(f"Sentence: {entry['sentence']}")
        print(f"Agents: {entry['agents']}")
        print(f"Results: {entry['results']}\n")


if __name__ == "__main__":
    classify_sentences()

Enter sentences (comma-separated):
Sentences: me gym ja k agaya

Specify agent(s) (e.g., sports, education or sports|fitness for multiple agents per sentence):
  - If same agent(s) for all, just provide one set (e.g., sports or sports|fitness)
  - If per-sentence agents, match count of agents to number of sentences (comma-separated)
Agent(s): fitness|marital


Sentence: me gym ja k agaya
Agents: fitness|marital
Results: {'fitness': 'interested', 'marital': 'uncertain'}



In [8]:
classify_sentences()

Enter sentences (comma-separated):
Sentences: mujhe gym nhi karna h

Specify agent(s) (e.g., sports, education or sports|fitness for multiple agents per sentence):
  - If same agent(s) for all, just provide one set (e.g., sports or sports|fitness)
  - If per-sentence agents, match count of agents to number of sentences (comma-separated)
Agent(s): fitness


Sentence: mujhe gym nhi karna h
Agents: fitness
Results: {'fitness': 'not_interested'}

